In [1]:
test_set_path = "../dataset/test-00000-of-00001.parquet"

In [2]:
import pandas as pd
test_df = pd.read_parquet(test_set_path)

In [3]:
from prompt.prompt import get_sentiment_classifier_prompt
from llm_service.llm_caller import LLMCaller
from config import LLMConfig, load_yaml_config
from utils.load_label_maps import load_label_maps
from utils.evaluate import get_accuracy
from utils.preprocess import *

import time
from tqdm import tqdm

In [4]:
def preprocess_pipelines(text: str) -> str:
    text = remove_url(text)
    text = remove_special_char(text)
    return text

In [5]:
LABEL_MAPS = load_label_maps("../dataset/label_maps.yaml")

In [6]:
load_yaml_config("../config/gpt_config.yaml")

In [7]:
llm_config = LLMConfig()
llm_service = LLMCaller(llm_config)

In [8]:
llm_config.MODEL

'gpt-3.5-turbo'

In [9]:
import os

total_row = len(test_df)
ans_df = pd.DataFrame(columns=["texts", "category", "process_time"])

RESULTS_DIR = "../results/"
results_path = os.path.join(RESULTS_DIR, f"{llm_service.llm_config.MODEL}_preprocess_results.csv")

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

for row in tqdm(range(total_row)):
    sentence = test_df.loc[row, "texts"]
    preprocessed_sentence = preprocess_pipelines(sentence)
    sentiment_classifier_prompt = get_sentiment_classifier_prompt(message=preprocessed_sentence)
    start_process = time.time()
    ans = llm_service.call(sentiment_classifier_prompt)
    process_time = time.time() - start_process
    try:
        category = LABEL_MAPS[ans]
    except KeyError:
        category = ans

    ans_df.loc[row, "texts"] = sentence
    ans_df.loc[row, "category"] = category
    ans_df.loc[row, "process_time"] = process_time

    ans_df.to_csv(results_path, index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2671/2671 [33:31<00:00,  1.33it/s]


In [10]:
acc = get_accuracy(test_df, ans_df)

In [11]:
acc

0.01759640584050917

In [12]:
post_process_ans_df = pd.DataFrame(columns=["texts", "category", "process_time"])
for row in range(len(ans_df)):
    category = ans_df.loc[row, "category"]
    if isinstance(category, str):
        if "neutral" in category:
            category = LABEL_MAPS["neutral"]
        elif "positive" in category:
            category = LABEL_MAPS["positive"]
        elif "negative" in category:
            category = LABEL_MAPS["negative"]
        elif "question" in category:
            category = LABEL_MAPS["question"]

    post_process_ans_df.loc[row, "texts"] = ans_df.loc[row, "texts"]
    post_process_ans_df.loc[row, "category"] = category
    post_process_ans_df.loc[row, "process_time"] = ans_df.loc[row, "process_time"]

In [13]:
acc = get_accuracy(test_df, post_process_ans_df)

In [14]:
acc

0.4780980906027705

In [15]:
for row in range(len(post_process_ans_df)):
    pred = post_process_ans_df.loc[row, "category"]
    actual = test_df.loc[row, "category"]
    text = post_process_ans_df.loc[row, "texts"]
    if pred != actual:
        print(f"{text}, actual: {actual}, pred: {pred}")

ทานได้ค่ะ น้ำซุป MK ต้มมาจากหัวผักกาด ซีอิ้วขาว เกลือ แลน้ำตาลค่ะ, actual: 1, pred: 0
เคล็ดลับที่ขาดไม่ได้ในการป้องกันผิวจากแสงแดด คือการทาครีมกันแดด สาวๆบ้างคนอาจจะคิดว่ามันไม่ใช่เรื่องสำคัญเท่าไหร่ แต่บอกเลยว่า ผิดมาก เพราะแสงแดดสมัยนี้แรงมาก และมีอนุภาพการทำลายผิวสูงมาก ถ้าไม่อยากให้ผิวเราถูกทำร้ายแบบซ้ำๆซาก ควรทาครีมกันแดดที่ดีมีคุณภาพอย่าง Eucerin Sun Age Repair Serum ปัจจุบันครีมกันแดดมีมากมายให้เราเลือกซื้อก็จริง แต่ก็อย่าลืมเลือกสิ่งที่ดีที่สุดให้ตัวเราอย่าง Eucerin Sun Acne oil Control ค่ะ, actual: 1, pred: 0
สำหรับสูตรของผม คือ Jack Daniel's หวานซ่อนเปรี้ยวชื่อ Jack Yellow Life ส่วนประกอบ Jack Daniel's 2 ชอท น้ำเก๊กฮวย น้ำมะนาว ใบมิ้นท์ เกลือเล็กน้อย วิธีปรุง นำ JackDaniel's มาเขย่ากับน้ำเก๊กฮวย เจือด้วยน้ำมะนาวบางๆ ตกแต่งด้วยใบมิ้น เสริฟด้วยแก้วที่ทาเกลือไว้ที่ปากแก้ว รสชาติที่จะได้คือหวาน หอม ซ่อนเปรี้ยว ด้วยคอนเซ็ปว่านี่แหละคือชีวิต จะหวานอย่างเดียวก็จะเลี่ยนไป จะเปรี้ยวเกินไปก็ไม่ใช่เรื่อง จึงควรจะมีทั้งเปรี้ยว ทั้งหวานคละเคล้ากันไปครับ, actual: 1, pred: 0
เจ้ว่าการ์นิเย่